
# Practicum II - Movie Database Applicaton
### Collaborators: Abasamia Akpan, Rebecca Brent and Alessia Pizzoccheri

This Notebook provides context, documentation and sample code for a Database Application using IMBd datasets; for clarity, this document has been divided into the following sections:

### 1. Introduction
### 2. Assumptions
### 3. Logical Models
### 4. Database and Relations Definition¶
### 1. Introduction
#### 1.1. Looking at the data
In this section, we analyze and deconstruct the various data sets found on the IMBd page; data is organized in seven TSV files, specifically:

Title AKAs
Title Basics
Title Episodes
Title Ratings
Title Crew
Name Basics
Title Principles Our first step is to establish relationships between each data set and identify a primary key (PK) on each table. Since no information was provided regarding the role of columns, we devised a solution to find unique identifiers within each data set; that is, for each table we compare the total number of rows against the total number of unique values for a given column.
1.2 Establishing PKs
Following the system aforementioned, we identified the following primary keys

## Notes


#### title_basics is fine but wont let me add a null key at the end to store the foreign keys in the other table that dont map to a primary key in title_basics table so i commented it out.  Loads 99 except for that%

#### title ratings has foreign keys that dont exist . I tried to mask them and they worked before but table insertion failed today- #### We could make a decision if we want to discard foreign keys tconst, parentTconst and nconst that dont map to a primary key in title_basics or name_basics table and consider them as bad data

#### title episodes has the same issue as above

#### title AKA's has the same issue as above

#### name basics is fine

#### principals is fine 

#### all the other tables should be working

#### we should probably chage some lengths of some columns  or do this if performance is a concern
#### https://stackoverflow.com/questions/8746207/1071-specified-key-was-too-long-max-key-length-is-1000-bytes
Have no idea how to implement though
.....

In [1]:

import pandas as pd
import numpy as np
import os
import sys
import pip
pip.main(['install','prettytable'])
pip.main(['install','mysql-connector-python'])
pip.main(['install','tqdm'])
pip.main(['install','pymysql'])

from tqdm import tqdm



Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import mysql.connector
from prettytable import PrettyTable 
from prettytable import from_csv
from mysql.connector import Error

def create_connection_nodb(host_name, user_name, user_password):
    """ create a connection to mysql host
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_database(connection, db_name):
    """ create a database 
    :param connection: Connection object
    :param db_name: name of database in form of string
    :return:
    """
    cursor = connection.cursor()
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + db_name
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def create_connection(host_name, user_name, user_password, db_name):
    """ create a connection to the (possibly newly created) database
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :param db_name: string name of database 
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        table_name = create_table_sql.split(" ")[5].split("(")[0]
        print("Successfully created table: " + table_name)
    except Error as e:
        print(e)
        
def main():
    
    database = "imbd_application"
    host_name = "localhost"
    user_name = "root"
    user_password = "Odyssey22"
    
    create_table_titlebasics = """CREATE TABLE IF NOT EXISTS titlebasics(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    titleType TEXT,
                                    primaryTitle TEXT,
                                    originalTitle TEXT,
                                    isAdult BOOLEAN,
                                    startYear DATE,
                                    endYear DATE,
                                    runTimeMinutes TIME)"""
    
    create_table_titleratings = """CREATE TABLE IF NOT EXISTS titleratings(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    averageRating FLOAT,
                                    numVotes INT,
                                    FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""


    create_table_titleepisodes = """CREATE TABLE IF NOT EXISTS titleepisodes(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    seasonNumber INT,
                                    episodeNumber INT,
                                    parentTconst CHAR(9),
                                    FOREIGN KEY (parentTconst) REFERENCES titlebasics(tconst))"""


    create_table_namebasics = """CREATE TABLE IF NOT EXISTS namebasics(
                                nconst VARCHAR(11) PRIMARY KEY,
                                primaryname TEXT,
                                birthyear DATE,
                                deathyear DATE,
                                noofmovies INT,
                                age INT,
                                currentdate DATE)"""
    
    
    create_table_genres = """CREATE TABLE IF NOT EXISTS genres(
                                genreID INT AUTO_INCREMENT  PRIMARY KEY,
                                genre VARCHAR(256) UNIQUE)"""
    
    create_table_knownfortitles = """CREATE TABLE IF NOT EXISTS knownfortitles(
                                knownfortitlesID INT AUTO_INCREMENT PRIMARY KEY,
                                titles VARCHAR(11) UNIQUE)"""
    
    create_table_primaryprofession = """CREATE TABLE IF NOT EXISTS primaryprofession(
                                primaryprofessionID INT AUTO_INCREMENT PRIMARY KEY,
                                profession VARCHAR(256) UNIQUE)"""
    
 
    
    
    create_table_genretitlebasics = """CREATE TABLE IF NOT EXISTS genretitlebasics(
                                titlebasicsID VARCHAR(11),
                                genreID INT,
                                PRIMARY KEY (titlebasicsID, genreID),
                                FOREIGN KEY (titlebasicsID) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (genreID) REFERENCES genres(genreID))"""
    

    
    create_table_primaryprofessiontitlenames = """CREATE TABLE IF NOT EXISTS primaryprofessiontitlenames(
                                primaryprofessionid INT,
                                namebasicsid VARCHAR(11),
                                PRIMARY KEY (primaryprofessionid, namebasicsid),
                                FOREIGN KEY (primaryprofessionid) REFERENCES primaryprofession(primaryprofessionid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst))"""
    
    create_table_knownfortitlenames = """CREATE TABLE IF NOT EXISTS knownfortitlenames(
                                namebasicsid VARCHAR(11),
                                knownfortitlesid INT,
                                PRIMARY KEY (namebasicsid, knownfortitlesid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst),
                                FOREIGN KEY (knownfortitlesid) REFERENCES knownfortitles(knownfortitlesid))"""
    
    conn = create_connection_nodb(host_name, user_name, user_password)
    
    # create database object if not already created and then connect the cursor to it
    if conn is not None:
        create_database(conn, database)
        conn = create_connection(host_name, user_name, user_password, database)
        
        # create tables
        if conn is not None:
            
            # create Title Basics table
            create_table(conn, create_table_titlebasics)

            # create Title Ratings table
            create_table(conn, create_table_titleratings)
            
            # create title episodes table
            create_table(conn, create_table_titleepisodes)
            
            # create name basics table
            create_table(conn, create_table_namebasics)
            
            # create genres table
            create_table(conn,create_table_genres)
            
            # create known for titles table
            create_table(conn,create_table_knownfortitles)
            
            # create primary profession table
            create_table(conn,create_table_primaryprofession)
            
            # create genre title basics table
            create_table(conn, create_table_genretitlebasics)

            # create Primary Profession Title Names table
            create_table(conn, create_table_primaryprofessiontitlenames)
            
            # create known for title names table
            create_table(conn, create_table_knownfortitlenames)
                
        else:
            print("Error! cannot create the database connection.")
    
    else:
        print("Error! Cannot create the MySQL connection.")
    
    


if __name__ == '__main__':
    main()


Connection to MySQL DB successful
Database created successfully
Connection to MySQL DB successful
Successfully created table: titlebasics
Successfully created table: titleratings
Successfully created table: titleepisodes
Successfully created table: namebasics
Successfully created table: genres
Successfully created table: knownfortitles
Successfully created table: primaryprofession
Successfully created table: genretitlebasics
Successfully created table: primaryprofessiontitlenames
Successfully created table: knownfortitlenames


## SQL communication helper ##

In [53]:
import sqlalchemy
database_username = 'root'
database_password = 'Odyssey22'
database_ip       = ''
database_name     = 'imbd_application'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [15]:
## Helper Functions

In [3]:
def load_dataframe(data_frame,table_name):
    data_frame_new=data_frame
    
    data_frame_new.to_sql(table_name,con=database_connection, if_exists='append',index=False)
    
    return 

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [73]:
def preprocess_dataframe_parsing(data_frame, column_name, final_column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(name=data_frame[column_name].str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    data_preprocessed_unique= pd.DataFrame(data_frame.name.unique())
    data_preprocessed_unique_column_rename=data_preprocessed_unique.rename(columns={0:final_column_name})
    
    return data_preprocessed_unique_column_rename

## Title Basics

In [71]:
# read title basic file
title_basics = pd.read_csv('title.basics.tsv',delimiter='\t',encoding='utf-8')
title_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [92]:
def preprocess_dataframe__for_genres(data_frame):
    df = data_frame
    lst_col = 'genres'
  
    df = df.drop(['titleType'], axis=1)
    df = df.drop(['primaryTitle'], axis=1)
    df = df.drop(['originalTitle'], axis=1)
    df=df.drop(['isAdult'], axis=1)
    df=df.drop(['startYear'], axis=1)
    df=df.drop(['endYear'], axis=1)
    df=df.drop(['runtimeMinutes'],axis=1)
    
    x = df.assign(**{lst_col:df[lst_col].str.split(',')})
    genres_df = pd.DataFrame({
                      col:np.repeat(x[col].values, x[lst_col].str.len())
                      for col in x.columns.difference([lst_col])
                      }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]

    return genres_df

df =  preprocess_dataframe__for_genres(title_basics)


genres_preprocessed=preprocess_dataframe_parsing(title_basics.genres, "genres", "genre")
genre_dict = genres_preprocessed['genre'].to_dict()
inv_map = {v: k for k, v in genre_dict.items()}
df = df.replace({"genres":inv_map})
df = df.rename(columns={"tconst" : "titlebasicsID", "genres" : "genreID"})
df.genreID = df.genreID + 1

In [ ]:
batch_size=100000
length=len(genres_preprocessed)//batch_size+1
for chunk in tqdm(batch(genres_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'genres')

In [98]:
batch_size = 100000
length = len(df)//batch_size+1

for chunk in tqdm(batch(df,batch_size),total=length):
    load_dataframe(chunk,'genretitlebasics')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.78it/s]


In [80]:
def preprocess_dataframe_title_basics(data_frame):
    title_basics_new_series=data_frame

    b = pd.to_datetime(title_basics_new_series['runtimeMinutes'], errors='coerce', format='%M')

    b.to_frame().runtimeMinutes
    df_1 = pd.DataFrame(dict(time_stamps = b)) 

    df_1['runtimeMinutes'] = df_1['time_stamps'].dt.minute
    title_basics_new_series=title_basics_new_series.drop(['runtimeMinutes'],axis=1)
    title_basics_new_series['runtimeMinutes']=df_1['runtimeMinutes']
    
    title_basics_new_frame_transpose=title_basics_new_series.astype({'isAdult': 'int32'})
    
    title_basics_new_frame_transpose["isAdult"]=title_basics_new_frame_transpose.isAdult.mask(title_basics_new_frame_transpose.isAdult > 1,1)

    title_basics_new_frame_transpose['startYear']= pd.to_datetime(title_basics.startYear, errors='coerce', format='%Y')

    
    title_basics_new_frame_transpose['endYear']= pd.to_datetime(title_basics.endYear, errors='coerce', format='%Y')
    title_basics_new_frame_transpose=title_basics_new_frame_transpose.drop(['genres'],axis=1)
    new_row = {
        'tconst':None, 
        'titleType':None, 
        'primaryTitle':None,
        'originalTitle':None,
        'isAdult':None, 
        'startYear':None,
        'endYear':None,
        'runtimeMinutes':None         
    } 
  
    title_basics_new_frame_transpose = title_basics_new_frame_transpose.append(new_row, ignore_index=True)
    title_basics_new_frame_transpose_new=title_basics_new_frame_transpose.rename(columns={"runtimeMinutes": "runTimeMinutes"})

    return title_basics_new_frame_transpose_new

title_basics_preprocessed=preprocess_dataframe_title_basics(title_basics)

In [7]:
len(title_basics_preprocessed)
title_basics_preprocessed

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runTimeMinutes
0,tt0000001,short,Carmencita,Carmencita,0,1894-01-01,NaT,1.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892-01-01,NaT,5.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892-01-01,NaT,4.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892-01-01,NaT,12.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893-01-01,NaT,1.0
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894-01-01,NaT,1.0
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894-01-01,NaT,1.0
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894-01-01,NaT,1.0
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894-01-01,NaT,45.0
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895-01-01,NaT,1.0


In [81]:
batch_size = 100000
length = len(title_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_basics_preprocessed, batch_size),total=length):
    load_dataframe(chunk,'titlebasics')

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


IntegrityError: (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry 'tt0000001' for key 'titlebasics.PRIMARY'
[SQL: INSERT INTO titlebasics (tconst, `titleType`, `primaryTitle`, `originalTitle`, `isAdult`, `startYear`, `endYear`, `runTimeMinutes`) VALUES (%(tconst)s, %(titleType)s, %(primaryTitle)s, %(originalTitle)s, %(isAdult)s, %(startYear)s, %(endYear)s, %(runTimeMinutes)s)]
[parameters: ({'tconst': 'tt0000001', 'titleType': 'short', 'primaryTitle': 'Carmencita', 'originalTitle': 'Carmencita', 'isAdult': 0, 'startYear': datetime.datetime(1894, 1, 1, 0, 0), 'endYear': None, 'runTimeMinutes': 1.0}, {'tconst': 'tt0000002', 'titleType': 'short', 'primaryTitle': 'Le clown et ses chiens', 'originalTitle': 'Le clown et ses chiens', 'isAdult': 0, 'startYear': datetime.datetime(1892, 1, 1, 0, 0), 'endYear': None, 'runTimeMinutes': 5.0}, {'tconst': 'tt0000003', 'titleType': 'short', 'primaryTitle': 'Pauvre Pierrot', 'originalTitle': 'Pauvre Pierrot', 'isAdult': 0, 'startYea

## Title ratings ##

In [36]:
# read data from title ratings file
title_ratings = pd.read_csv('/Users/abasiamaakpan/Downloads/title.ratings.tsv', delimiter='\t',encoding='utf-8')
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1656
1,tt0000002,6.1,201
2,tt0000003,6.5,1369
3,tt0000004,6.2,122
4,tt0000005,6.2,2151
...,...,...,...
1086697,tt9916580,7.2,5
1086698,tt9916690,6.6,5
1086699,tt9916720,6.0,65
1086700,tt9916766,6.9,14


In [469]:
def preprocess_dataframe_title_ratings(data_frame):
    title_ratings_new_series=data_frame
    
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'averageRating': 'float'})
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'numVotes': 'int32'})
    #title_ratings_preprocessed.tconst = title_ratings_preprocessed.tconst.mask(~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)
    return title_ratings_new_frame_avg_rating
title_ratings_preprocessed=preprocess_dataframe_title_ratings(title_ratings)

In [458]:
len(title_ratings_preprocessed)

1086702

In [464]:
#title_ratings_preprocessed[~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst)]

title_ratings_preprocessed.tconst = title_ratings_preprocessed.tconst.mask(~title_ratings_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)



,tconst,averageRating,numVotes


In [470]:
batch_size = 100000
length = len(title_ratings_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_ratings_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleratings')

 91%|█████████ | 10/11 [00:40<00:04,  4.09s/it]


IntegrityError: (mysql.connector.errors.IntegrityError) 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`last_trial_for_the_night`.`titleratings`, CONSTRAINT `titleratings_ibfk_1` FOREIGN KEY (`tconst`) REFERENCES `titlebasics` (`tconst`))
[SQL: INSERT INTO titleratings (tconst, `averageRating`, `numVotes`) VALUES (%(tconst)s, %(averageRating)s, %(numVotes)s)]
[parameters: ({'tconst': 'tt7348840', 'averageRating': 6.2, 'numVotes': 10}, {'tconst': 'tt7348842', 'averageRating': 6.1, 'numVotes': 10}, {'tconst': 'tt7348844', 'averageRating': 6.0, 'numVotes': 9}, {'tconst': 'tt7348952', 'averageRating': 6.9, 'numVotes': 9}, {'tconst': 'tt7348976', 'averageRating': 6.6, 'numVotes': 8}, {'tconst': 'tt7348978', 'averageRating': 6.8, 'numVotes': 6}, {'tconst': 'tt7349002', 'averageRating': 7.1, 'numVotes': 8}, {'tconst': 'tt7349004', 'averageRating': 6.8, 'numVotes': 6}  ... displaying 10 of 86702 total bound parameter sets ...  {'tconst': 'tt9916766', 'averageRa

## Title Episodes ##

In [42]:
# read title episodes data
title_episodes = pd.read_csv('/Users/abasiamaakpan/Downloads/title.episode.tsv', delimiter='\t',encoding='utf-8')
title_episodes

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16
...,...,...,...,...
5252722,tt9916846,tt1289683,3,18
5252723,tt9916848,tt1289683,3,17
5252724,tt9916850,tt1289683,3,19
5252725,tt9916852,tt1289683,3,20


In [490]:
def preprocess_dataframe_title_episodes(data_frame):
    title_episodes_new_series=data_frame
    title_episodes_coerce_seasonNum = pd.to_numeric(title_episodes_new_series['seasonNumber'], errors='coerce')
    title_episodes_coerce_episodeNum = pd.to_numeric(title_episodes_new_series['episodeNumber'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_episodes_coerce_seasonNum)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_episodes_coerce_episodeNum)) 

    df_1['seasonNumber'] = df_1['test_column_1']
    df_2['episodeNumber'] = df_2['test_column_2']
    
    title_episodes_new_frame=title_episodes_new_series.drop(['seasonNumber'],axis=1)
    title_episodes_new_frame=title_episodes_new_series.drop(['episodeNumber'],axis=1)
    
    title_episodes_new_frame['seasonNumber']=df_1['seasonNumber']
    title_episodes_new_frame['episodeNumber']=df_2['episodeNumber']
    return title_episodes_preprocessed

title_episodes_preprocessed = preprocess_dataframe_title_episodes(title_episodes)
title_episodes_preprocessed

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3.0,42.0
4,tt0043631,tt0989125,2.0,16.0
...,...,...,...,...
5252722,tt9916846,tt1289683,3.0,18.0
5252723,tt9916848,tt1289683,3.0,17.0
5252724,tt9916850,tt1289683,3.0,19.0
5252725,tt9916852,tt1289683,3.0,20.0


In [491]:
title_episodes_preprocessed[~title_episodes_preprocessed.parentTconst.isin(title_basics_preprocessed.tconst)]



,tconst,parentTconst,seasonNumber,episodeNumber


In [492]:
title_episodes_preprocessed.tconst = title_episodes_preprocessed.tconst.mask(~title_episodes_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)



In [ ]:
title_episodes_preprocessed[~title_episodes_preprocessed.parentTconst.isin(title_basics_preprocessed.tconst)]



In [493]:
batch_size = 100000
length = len(title_episodes_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_episodes_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleepisodes')

  0%|          | 0/53 [00:03<?, ?it/s]


IntegrityError: (mysql.connector.errors.IntegrityError) 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`last_trial_for_the_night`.`titleepisodes`, CONSTRAINT `titleepisodes_ibfk_1` FOREIGN KEY (`parentTconst`) REFERENCES `titlebasics` (`tconst`))
[SQL: INSERT INTO titleepisodes (tconst, `parentTconst`, `seasonNumber`, `episodeNumber`) VALUES (%(tconst)s, %(parentTconst)s, %(seasonNumber)s, %(episodeNumber)s)]
[parameters: ({'tconst': 'tt0041951', 'parentTconst': 'tt0041038', 'seasonNumber': 1.0, 'episodeNumber': 9.0}, {'tconst': 'tt0042816', 'parentTconst': 'tt0989125', 'seasonNumber': 1.0, 'episodeNumber': 17.0}, {'tconst': 'tt0042889', 'parentTconst': 'tt0989125', 'seasonNumber': None, 'episodeNumber': None}, {'tconst': 'tt0043426', 'parentTconst': 'tt0040051', 'seasonNumber': 3.0, 'episodeNumber': 42.0}, {'tconst': 'tt0043631', 'parentTconst': 'tt0989125', 'seasonNumber': 2.0, 'episodeNumber': 16.0}, {'tconst': 'tt0043693', 'parentTconst': 'tt0989125

## Name Basics

In [13]:
name_basics = pd.read_csv('/Users/abasiamaakpan/Downloads/name.basics.tsv', delimiter='\t',encoding='utf-8')
name_basics

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0053137,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0059956,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0072562,tt0080455,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0050976,tt0083922,tt0050986"
...,...,...,...,...,...,...
10464396,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10464397,nm9993716,Essias Loberg,\N,\N,NaN,\N
10464398,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10464399,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


In [14]:
def preprocess_dataframe_name_basics(data_frame):
    name_basics_new_series=data_frame
    name_basics_new_series=name_basics_new_series.drop(['primaryProfession', 'knownForTitles'],axis=1)
    
    name_basics_new_series['birthYear']= pd.to_datetime(name_basics_new_series.birthYear, errors='coerce', format='%Y')

    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]
    
    name_basics_new_series['deathYear']= pd.to_datetime(name_basics_new_series.deathYear, errors='coerce', format='%Y')
    
    return name_basics_new_series

name_basics_preprocessed = preprocess_dataframe_name_basics(name_basics)

name_basics_preprocessed["noofmovies"] = ""
name_basics_preprocessed["age"] = ""
name_basics_preprocessed["currentdate"] = ""

In [15]:
name_basics_preprocessed_new= pd.to_numeric(name_basics_preprocessed['noofmovies'], errors='coerce')
name_basics_preprocessed_new = pd.to_numeric(name_basics_preprocessed['age'], errors='coerce')
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed.currentdate, errors='coerce', format='%Y')


In [16]:
df_1 = pd.DataFrame(dict(test_column_1 = name_basics_preprocessed_new)) 
df_2 = pd.DataFrame(dict(test_column_2 = name_basics_preprocessed_new)) 

df_1['noofmovies'] = df_1['test_column_1']
df_2['age'] = df_2['test_column_2']

name_basics_preprocessed['noofmovies']=df_1['noofmovies']
name_basics_preprocessed['age']=df_2['age']
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed['currentdate'])

In [17]:
len(name_basics_preprocessed)

10464401

In [18]:
name_basics_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10464401 entries, 0 to 10464400
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   nconst       object        
 1   primaryName  object        
 2   birthYear    datetime64[ns]
 3   deathYear    datetime64[ns]
 4   noofmovies   float64       
 5   age          float64       
 6   currentdate  datetime64[ns]
dtypes: datetime64[ns](3), float64(2), object(2)
memory usage: 558.9+ MB


## Approach 1- With sql alchemy ##

In [505]:
batch_size = 100000
length = len(name_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(name_basics_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'namebasics')

100%|██████████| 105/105 [04:25<00:00,  2.53s/it]


In [57]:
# UNCOMMENT TO DISPLAY TABLE ROWS
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="alupizzo92",
#     database="imbd_application"
# )
# mycursor = mydb.cursor()

# query = "SELECT * FROM namebasics LIMIT 5;"
# mycursor.execute(query)

## Genres ##

In [32]:
genres_preprocessed=preprocess_dataframe_parsing(title_basics.genres, "genres", "genre")

In [284]:
genres_preprocessed

,genres
0,Documentary
1,Short
2,Animation
3,Comedy
4,Romance
5,Sport
6,Action
7,News
8,Drama
9,Fantasy


In [75]:
len(genres_preprocessed)

11583271

In [ ]:
batch_size=100000
length=len(genres_preprocessed)//batch_size+1
for chunk in tqdm(batch(genres_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'genres')

## Known for titles ##

In [285]:
knownForTitles_preprocessed=preprocess_dataframe_parsing(name_basics.knownForTitles, "knownForTitles", "titles")

In [286]:
knownForTitles_preprocessed

,knownForTitles
0,tt0031983
1,tt0072308
2,tt0053137
3,tt0050419
4,tt0037382
...,...
1583516,tt8743558
1583517,tt8743742
1583518,tt8187876
1583519,tt8744074


In [ ]:
batch_size=100000
length=len(knownForTitles_preprocessed)//batch_size+1
for chunk in tqdm(batch(knownForTitles_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'knownFortitles')

## primary profession ##

In [287]:
pprofession_preprocessed=preprocess_dataframe_parsing(name_basics.primaryProfession, "primaryProfession", "profession")

In [288]:
pprofession_preprocessed

,primaryProfession
0,soundtrack
1,actor
2,miscellaneous
3,actress
4,music_department
5,writer
6,director
7,producer
8,make_up_department
9,composer


In [ ]:
batch_size=100000
length=len(pprofession_preprocessed)//batch_size+1
for chunk in tqdm(batch(pprofession_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'primaryprofession')

## Genre Title Basics-below functions work for the rest of the tables. Havent refactored all the tables yet

In [33]:
def createAuxilliaryTables(Frame1,Frame2, column_rename_1, column_rename_2):
    
    Frame3 = pd.merge(Frame1, Frame2, on=Frame1.index, how='outer')
    Frame4=Frame3.rename(columns={"key_0": column_rename_1})
    created_table = pd.merge(Frame4, Frame2, on=Frame4.index, how='outer')
    created_table=created_table.rename(columns={"key_0": column_rename_2})
        
    return created_table

In [34]:
def concat_tables (Frame1,Frame2,column1_name,column2_name,column1_name_new,column2_name_new):
    NewFrame_column1=createAuxilliaryTables(Frame1, Frame1, column1_name, column1_name_new)
    NewFrame_column2=createAuxilliaryTables(Frame2, Frame2, column2_name, column2_name_new)
    NewFrame_column2=NewFrame_column2.drop([column2_name_new],axis=1)
    NewFrame_column1=NewFrame_column1.drop([column1_name_new],axis=1)
    NewFrame_final_table = pd.concat([NewFrame_column2, NewFrame_column1], ignore_index=True)
    
    return NewFrame_final_table


In [35]:
title_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
7292276,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
7292277,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
7292278,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
7292279,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [40]:
title_basics_preprocessed_new_1=title_basics.drop(['tconst'],axis=1)
title_basics_preprocessed_new_2=title_basics_preprocessed_new_1.drop(['titleType'],axis=1)
title_basics_preprocessed_new_3=title_basics_preprocessed_new_2.drop(['primaryTitle'],axis=1)
title_basics_preprocessed_new_4=title_basics_preprocessed_new_3.drop(['originalTitle'],axis=1)
title_basics_preprocessed_new_5=title_basics_preprocessed_new_4.drop(['isAdult'],axis=1)
title_basics_preprocessed_new_6=title_basics_preprocessed_new_5.drop(['startYear'],axis=1)
title_basics_preprocessed_new_7=title_basics_preprocessed_new_6.drop(['endYear'],axis=1)
title_basics_preprocessed_new_8=title_basics_preprocessed_new_7.drop(['runtimeMinutes'],axis=1)
title_basics_preprocessed_new_9=title_basics_preprocessed_new_8.drop(['genres'],axis=1)



title_basics_preprocessed_new_9

""
0
1
2
3
4
...
7292276
7292277
7292278
7292279


In [41]:
genres_preprocessed

,genre
0,Documentary
1,Short
2,Animation
3,Comedy
4,Romance
5,Sport
6,Action
7,News
8,Drama
9,Fantasy


In [43]:
genres_preprocessed_new_1=genres_preprocessed.drop(['genre'],axis=1)

genres_preprocessed_new_1

""
0
1
2
3
4
5
6
7
8
9


In [45]:
genre_title_basics_final_table=concat_tables(title_basics_preprocessed_new_9,genres_preprocessed_new_1,"titilebasicsID","genreID","titlebasicsNew", "genreNew")

genre_title_basics_final_table

,genreID,titilebasicsID
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
...,...,...
7292306,NaN,7292276.0
7292307,NaN,7292277.0
7292308,NaN,7292278.0
7292309,NaN,7292279.0


In [ ]:
batch_size=100000
length=len(genre_title_basics_final_table)//batch_size+1
for chunk in tqdm(batch(genre_title_basics_final_table,batch_size),total=length):
    load_dataframe(chunk,'genretitlebasics')

## Primaryprofessiontitlenames

In [355]:
pprofession_preprocessed

,primaryProfession
0,soundtrack
1,actor
2,miscellaneous
3,actress
4,music_department
5,writer
6,director
7,producer
8,make_up_department
9,composer


In [374]:
primaryprofessiontitlenames= pd.DataFrame(pprofession_preprocessed)
primaryprofessiontitlenames

,primaryProfession
0,soundtrack
1,actor
2,miscellaneous
3,actress
4,music_department
5,writer
6,director
7,producer
8,make_up_department
9,composer


In [377]:
primaryprofessiontitlenames_new_unique=primaryprofessiontitlenames.drop("primaryProfession",axis=1)

primaryprofessiontitlenames_new_unique

""
0
1
2
3
4
5
6
7
8
9


In [378]:
name_basics_preprocessed_new=name_basics_preprocessed.drop("nconst",axis=1)
#name_basics_preprocessed_new=name_basics_preprocessed_new.drop("primaryName",axis=1)
name_basics_preprocessed_new=name_basics_preprocessed_new.drop("birthYear",axis=1)
name_basics_preprocessed_new=name_basics_preprocessed_new.drop("deathYear",axis=1)
name_basics_preprocessed_new=name_basics_preprocessed_new.drop("noofmovies",axis=1)
name_basics_preprocessed_new=name_basics_preprocessed_new.drop("age",axis=1)
name_basics_preprocessed_new_unique=name_basics_preprocessed_new.drop("currentdate",axis=1)
#name_basics_preprocessed_new_unique= pd.DataFrame(name_basics_preprocessed_new.primaryName.unique())



In [379]:
name_basics_preprocessed_new_unique=name_basics_preprocessed_new_unique.rename(columns={0: "primaryName"})
name_basics_preprocessed_new_unique

,primaryName
0,Fred Astaire
1,Lauren Bacall
2,Brigitte Bardot
3,John Belushi
4,Ingmar Bergman
...,...
10464396,Romeo del Rosario
10464397,Essias Loberg
10464398,Harikrishnan Rajan
10464399,Aayush Nair


In [380]:
name_basics_preprocessed_unique_new=name_basics_preprocessed_new_unique.drop("primaryName",axis=1)
name_basics_preprocessed_unique_new

""
0
1
2
3
4
...
10464396
10464397
10464398
10464399


In [381]:
name_basics_preprocess_column1=createAuxilliaryTables(primaryprofessiontitlenames_new_unique, primaryprofessiontitlenames_new_unique, "primaryprofessionId", "primaryprofessionIdNew")
name_basics_preprocess_column1

,primaryprofessionIdNew,primaryprofessionId
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5
6,6,6
7,7,7
8,8,8
9,9,9


In [382]:
name_basics_preprocess_column2=createAuxilliaryTables(name_basics_preprocessed_unique_new, name_basics_preprocessed_unique_new, "namebasicsId", "namebasicsIdNew")
name_basics_preprocess_column2

,namebasicsIdNew,namebasicsId
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
10464396,10464396,10464396
10464397,10464397,10464397
10464398,10464398,10464398
10464399,10464399,10464399


In [383]:
name_basics_preprocess_column2=name_basics_preprocess_column2.drop(['namebasicsIdNew'],axis=1)
name_basics_preprocess_column1=name_basics_preprocess_column1.drop(['primaryprofessionIdNew'],axis=1)



In [384]:
primaryprofession_final_table = pd.concat([name_basics_preprocess_column2, name_basics_preprocess_column1], ignore_index=True)

In [391]:
primaryprofession_final_table.tail()

,namebasicsId,primaryprofessionId
10464437,NaN,36.0
10464438,NaN,37.0
10464439,NaN,38.0
10464440,NaN,39.0
10464441,NaN,40.0


In [ ]:
batch_size=100000
length=len(primaryprofession_final_table)//batch_size+1
for chunk in tqdm(batch(primaryprofession_final_table),total=length):
    load_dataframe(chunk,'primaryprofessiontitlenames')

## Known for Title Names

In [354]:
knownForTitles_preprocessed

,knownForTitles
0,tt0031983
1,tt0072308
2,tt0053137
3,tt0050419
4,tt0037382
...,...
1583516,tt8743558
1583517,tt8743742
1583518,tt8187876
1583519,tt8744074


In [388]:
knownForTitles_preprocessed_new=knownForTitles_preprocessed.drop(['knownForTitles'],axis=1)



In [389]:
knownForTitles_preprocessed_new

""
0
1
2
3
4
...
1583516
1583517
1583518
1583519


In [366]:
name_basics_preprocessed_unique_new

""
0
1
2
3
4
...
10464396
10464397
10464398
10464399


In [395]:
knownForTitleNames_preprocess_column1=createAuxilliaryTables(knownForTitles_preprocessed_new, knownForTitles_preprocessed_new, "knownForTitlesId", "knownForTitlesNew")

knownForTitleNames_preprocess_column2=createAuxilliaryTables(name_basics_preprocessed_unique_new, name_basics_preprocessed_unique_new, "namebasicsId", "namebasicsNew")

knownForTitleNames_preprocess_column1

,knownForTitlesNew,knownForTitlesId
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
1583516,1583516,1583516
1583517,1583517,1583517
1583518,1583518,1583518
1583519,1583519,1583519


In [396]:
knownForTitleNames_preprocess_column2=knownForTitleNames_preprocess_column2.drop(['namebasicsNew'],axis=1)
knownForTitleNames_preprocess_column1=knownForTitleNames_preprocess_column1.drop(['knownForTitlesNew'],axis=1)
knownForTitleNames_final_table = pd.concat([knownForTitleNames_preprocess_column2, knownForTitleNames_preprocess_column1], ignore_index=True)
knownForTitleNames_final_table

,namebasicsId,knownForTitlesId
0,0.0,NaN
1,1.0,NaN
2,2.0,NaN
3,3.0,NaN
4,4.0,NaN
...,...,...
12047917,NaN,1583516.0
12047918,NaN,1583517.0
12047919,NaN,1583518.0
12047920,NaN,1583519.0


In [ ]:
batch_size=100000
length=len(knownForTitleNames_final_table)//batch_size+1
for chunk in tqdm(batch(knownForTitleNames_final_table,batch_size),total=length):
    load_dataframe(chunk,'knownfortitlenames')

## Question 5

In [ ]:
conn = create_connection("localhost", "root", "Odyssey22", "imbd_application")
c = conn.cursor()

c.execute("DROP TABLE imbd_application")


In [ ]:
conn = create_connection("localhost", "root", "Odyssey22", "imbd_application")
c = conn.cursor()

operation = """SELECT COUNT(KTN.knownfortitlesID)
               FROM namebasics AS NB, knownfortitlenames AS KTN
               WHERE KTN.knownfortitlesID IN (SELECT KT.knownfortitlesID 
                                              FROM knownforTitles AS KT 
                                              WHERE KT.titles IN (SELECT tconst
                                                                  FROM titlebasics AS TB 
                                                                  WHERE TB.titletype = 'Movie'))
                GROUP BY namebasicsID
                LIMIT 5"""

for result in c.execute(operation, multi=True):
    if result.with_rows:
        print("Rows produced by statement '{}':".format(
          result.statement))
        print(result.fetchall())
    else:
        print("Number of rows affected by statement '{}': {}".format(
          result.statement, result.rowcount))